In [ ]:
1. 定义数据

In [81]:
def createDataSet():
    dataset = [[0, 0, 0, 0, 'N'], 
               [0, 0, 0, 1, 'N'], 
               [1, 0, 0, 0, 'Y'], 
               [2, 1, 0, 0, 'Y'], 
               [2, 2, 1, 0, 'Y'], 
               [2, 2, 1, 1, 'N'], 
               [1, 2, 1, 1, 'Y']]
    labels = ['outlook', 'temperature', 'humidity', 'windy']
    return dataset, labels


def createDataSet2():
    dataSet = [[1, 1, 'yes'],
               [1, 1, 'yes'],
               [1, 0, 'no'],
               [0, 1, 'no'],
               [0, 1, 'no']]
    labels = ['no surfacing','flippers']
    return dataSet, labels


In [82]:
dataSet,labels = createDataSet2()
print(dataSet)
print(labels)

[[1, 1, 'yes'], [1, 1, 'yes'], [1, 0, 'no'], [0, 1, 'no'], [0, 1, 'no']]
['no surfacing', 'flippers']


2. 计算熵

In [83]:
from math import log

# unique value
def unique(dataset):
    labelCounts = {}
    for featVec in dataset: #the the number of unique elements and their occurance
        currentLabel = featVec[-1]
        if currentLabel not in labelCounts.keys(): labelCounts[currentLabel] = 0
        labelCounts[currentLabel] += 1
    
    return labelCounts

print(unique(dataSet))

def entropy(dataset):
    numEntries = len(dataset)  
    shannonEnt = 0.0
    labelCounts = unique(dataset)
    for key in labelCounts:
        prob = float(labelCounts[key])/numEntries
        shannonEnt -= prob * log(prob,2) #log base 2
    return shannonEnt

print(entropy(dataSet))

{'yes': 2, 'no': 3}
0.9709505944546686


3. 选择最佳特征

In [84]:
# 根据指定的特征来分裂数据集
# dataSet:数据集（MxN），axis：特征的索引，即第几个特征：，value：所选特征的取值
# 返回一个数据集，该数据集以axis索引表示的特征为分裂特征，并且该分裂特征的值为value时得到的。

# @sheldowong 它的做法是，在分裂时，将分裂特征对应的列去掉，按照分裂特征的取值划分数据集，也就是，每次划分以后，特征就会减少一列
def splitDataSet(dataSet, axis, value):
    retDataSet = []
    for featVec in dataSet:
        if featVec[axis] == value:
            reducedFeatVec = featVec[:axis]     #chop out axis used for splitting
            reducedFeatVec.extend(featVec[axis+1:])
            retDataSet.append(reducedFeatVec)
    return retDataSet
# 返回分裂特征的索引
print('以第0个特征为分裂特征进行分裂数据集，')
print('分裂特征值为0的子集合：',splitDataSet(dataSet,0,0))
print('分裂特征值为1的子集合：',splitDataSet(dataSet,0,1))
print('分裂特征值为2的子集合：',splitDataSet(dataSet,0,2))

以第0个特征为分裂特征进行分裂数据集，
分裂特征值为0的子集合： [[1, 'no'], [1, 'no']]
分裂特征值为1的子集合： [[1, 'yes'], [1, 'yes'], [0, 'no']]
分裂特征值为2的子集合： []


In [85]:
def chooseBestFeatureToSplit(dataSet):
    # 特征个数
    numFeatures = len(dataSet[0]) - 1# 数据集中元素的最后一列为类别标签，所以需减1
    # 信息熵
    baseEntropy = entropy(dataSet)
    # 信息增益
    bestInfoGain = 0.0; bestFeature = -1# 初始化
   
    #遍历特征
    for i in range(numFeatures):
        featList = [element[i] for element in dataSet]# 得到数据集中第i个特征的所有取值
        uniqueVals = set(featList)# 对featList去重,得到第i个特征的特征值集合
        newEntropy = 0.0
        
        for value in uniqueVals:
            subDataSet = splitDataSet(dataSet,i,value)
            prob = len(subDataSet)/float(len(dataSet))
            newEntropy += prob * entropy(subDataSet)
        infoGain = baseEntropy - newEntropy
        
        if(infoGain > bestInfoGain):
            bestInfoGain = infoGain
            bestFeature = i
    return bestFeature
bestFeature = chooseBestFeatureToSplit(dataSet)
print(labels[bestFeature])

print( '分裂特征的索引为：',bestFeature)

no surfacing
分裂特征的索引为： 0


4. 根据最佳特征划分数据

5. 构建决策树（1. 多重字典 2. 树结构）

In [86]:
import operator
 
#多数表决
def majorityCnt(classList):
    classCount = {}
    for vote in classList:
        if vote not in classCount.keys(): classCount[vote] = 0
        classCount[vote] += 1
    sortedClassCount = sorted(classCount.iteritems(),key=operator.itemgetter(1),reverse=True)
    return sortedClassCount[0][0]

In [87]:
# 创建决策树
def createTree(dataSet,labels):
    classList = [example[-1] for example in dataSet]
    
    ### 终止条件
    if classList.count(classList[0]) == len(classList): 
        return classList[0]# 类别完全相同则停止继续划分
    
    if len(dataSet[0]) == 1: # 遍历完所有特征时返回出现次数最多的
        return majorityCnt(classList)
    
    ### 选择最佳特征，划分数据集
    bestFeat = chooseBestFeatureToSplit(dataSet)

    bestFeatLabel = labels[bestFeat]
    myTree = {bestFeatLabel:{}}
    del(labels[bestFeat])
    
    
    featValues = [example[bestFeat] for example in dataSet]
    uniqueVals = set(featValues)
    for value in uniqueVals:
        subLabels = labels[:]   #copy all of labels, so trees don't mess up existing labels
        print(subLabels)
        myTree[bestFeatLabel][value] = createTree(splitDataSet(dataSet, bestFeat, value),subLabels)
    return myTree 

myTree= createTree(dataSet,labels)

import json
print(json.dumps(myTree,indent=4))

['flippers']
['flippers']
[]
[]
{
    "no surfacing": {
        "0": "no",
        "1": {
            "flippers": {
                "0": "no",
                "1": "yes"
            }
        }
    }
}
